In [1]:
import cv2
from utils import *

In [3]:
path = '/Users/emmanuelazuh/Downloads/trainset/'
zebs = ['akn.031.029.left.avi', 'akn.031.037.left.avi', 'akn.088.141.left.avi', 
        'akn.174.086.left.avi', 'akn.174.090.left.avi', 'akn.174.124.left.avi', 
        'akn.179.178.left.avi', 'akn.184.139.left.avi', 'akn.199.020.left.avi', 
        'akn.204.006.left.avi', 'akn.211.002.left.avi', 'akn.211.006.left.avi', 
        'akn.212.134.left.avi', 'akn.212.179.left.avi', 'akn.212.221.left.avi', 
        'akn.212.237.left.avi', 'akn.215.175.left.avi', 'akn.228.225.left.avi', 
        'akn.228.244.left.avi', 'akn.243.043.left.avi', 'akn.244.141.left.avi', 
        'akn.247.011.left.avi', 'akn.250.300.left.avi', 'akn.263.136.left.avi', 
        'akn.266.024.left.avi', 'akn.266.396.left.avi', 'akn.266.430.left.avi', 
        'akn.267.004.left.avi', 'akn.267.064.left.avi', 'akn.267.135.left.avi', 
        'akn.269.036.left.avi', 'akn.269.094.left.avi', 'akn.269.111.left.avi', 
        'akn.270.197.left.avi', 'akn.271.109.left.avi', 'akn.272.018.left.avi', 
        'akn.273.014.left.avi', 'akn.273.056.left.avi', 'akn.273.074.left.avi', 
        'akn.275.015.left.avi', 'akn.275.115.left.avi', 'akn.279.026.left.avi', 
        'akn.280.006.left.avi', 'akn.281.131.left.avi', 'akn.282.083.left.avi', 
        'akn.283.065.left.avi', 'akn.283.175.left.avi', 'akn.289.008.left.avi', 
        'akn.289.069.left.avi', 'akn.292.154.left.avi', 'akn.294.156.left.avi', 
        'akn.295.027.left.avi', 'akn.393.054.left.avi']

In [4]:
def select_white_yellow(image):
    converted = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
    # white color mask
    lower = np.uint8([  0, 200,   0])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(converted, lower, upper)
    # yellow color mask
    lower = np.uint8([ 10,   0, 100])
    upper = np.uint8([ 40, 255, 255])
    yellow_mask = cv2.inRange(converted, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    return cv2.bitwise_and(image, image, mask = mask)

def select_rgb_white_yellow(image): 
    # white color mask
    lower = np.uint8([200, 200, 200])
    upper = np.uint8([255, 255, 255])
    white_mask = cv2.inRange(image, lower, upper)
    # yellow color mask
    lower = np.uint8([190, 190,   0])
    upper = np.uint8([255, 255, 255])
    yellow_mask = cv2.inRange(image, lower, upper)
    # combine the mask
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked = cv2.bitwise_and(image, image, mask = mask)
    return masked

def convert_gray_scale(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

In [18]:
"""Zebra!!!"""
ctr = 0
vid_idx = 0
cap = cv2.VideoCapture(path + zebs[vid_idx])
while(cap.isOpened()):
    ret, frame = cap.read()

    if (ret): # and (ctr % 2 == 0):
        h,w,l = frame.shape
        h,w = h/2, w/2
        
        # Use grayscale, blur and compute edges
        gray_image = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        blurred_image = cv2.GaussianBlur(gray_image, (15, 15), 0)
        
#         dil_kernel = np.ones((5,5),np.uint8)
#         erosion = cv2.erode(blurred_image,dil_kernel,iterations = 10)
#         dilate = cv2.dilate(erosion,dil_kernel,iterations = 10)
        
        filtered_image = cv2.bilateralFilter(blurred_image, 11, 17, 17)
        edge_image = cv2.Canny(blurred_image, 10, 30)
        region_of_interest = select_region(edge_image)
        
        _, contours, _ = cv2.findContours(region_of_interest, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
#         draw_cnts = np.array([])
#         for cnt in contours:
#             peri = cv2.arcLength(cnt, True)
#             approx = cv2.approxPolyDP(cnt, 0.04 * peri, True)
#             draw_cnts = np.append(draw_cnts, approx)
#         perimeters = list(map(lambda cnt: cv2.arcLength(cnt,True), contours))
        
        possible_zebra = filter(lambda cnt: cv2.arcLength(cnt, False) > 80, contours)
        im = cv2.drawContours(region_of_interest, pos, -1, (255, 255, 0), 5)
        cv2.imshow('frame',cv2.resize(im, (w,h)))
#         cv2.imshow('frame',cv2.resize(frame, (w,h)))
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
    ctr += 1
            
# cap.release()
cv2.destroyAllWindows()

In [ ]:
"""
Notes for zebra:

"""

In [17]:
def merge(filenames):
    # bridge-0, city_entry-1, city_exit-2, bump-3, wipers-4, zebra-5
    path = '/Users/emmanuelazuh/gitdir/visionhack/'
    a = []
    for _file in filenames:
        print path + _file
        files_and_tags = read_files_and_tags(path + _file)
        a.append(list(map(lambda x: x[1], files_and_tags)))
    files = list(map(lambda x: x[0] + ' ', files_and_tags))
    
    for i in range(len(a)):
        for j in range(len(a[0])):
            files[j] += a[i][j][i]
    with open('final_submission.txt', 'w') as s:
        s.write('\n'.join(files))

In [3]:
files_and_tags = read_files_and_tags('bridge.txt')
files = list(map(lambda x: x[0], files_and_tags))
for i in files:
    with open('generic.txt', 'a+') as out:
            out.write(i + ' 000000\n')
    

In [19]:
list_files = ['bridge.txt', 'generic.txt', 'generic.txt', 
              'generic.txt', 'generic.txt', 'zebras.txt']
merge(list_files)

/Users/emmanuelazuh/gitdir/visionhack/bridge.txt
/Users/emmanuelazuh/gitdir/visionhack/generic.txt
/Users/emmanuelazuh/gitdir/visionhack/generic.txt
/Users/emmanuelazuh/gitdir/visionhack/generic.txt
/Users/emmanuelazuh/gitdir/visionhack/generic.txt
/Users/emmanuelazuh/gitdir/visionhack/zebras.txt
